In [81]:
import pandas as pd
import numpy as np

In [142]:
df = pd.read_csv(r'L:\T2040\soundcast_2014\outputs\daysim\_trip.tsv', sep='\t')

In [143]:
df['person_id'] = df['hhno'].astype('str') + '_' + df['pno'].astype('str')

In [234]:
activity = pd.DataFrame()
activity.index = [[],[]]

for person_id in df['person_id'].unique()[0:500]:
    trip_subsample = df[df['person_id'] == person_id]
    for row in xrange(0, len(trip_subsample)-1):
        row_index = 0
        current_row = trip_subsample.iloc[row]
        next_row = trip_subsample.iloc[row+1]

        # Activity from midnight until first trip
        if row == 0:
            activity.ix[(person_id,0),'person_id'] = person_id
            activity.ix[(person_id,0),'activity'] = current_row['opurp']
            activity.ix[(person_id,0),'activity_index'] = 0
            activity.ix[(person_id,0),'parcel'] = current_row['opcl']
            activity.ix[(person_id,0),'begin_time'] = 0 # minutes after midnight
            activity.ix[(person_id,0),'end_time'] = current_row['deptm']
            activity.ix[(person_id,0),'duration'] = activity.ix[(person_id,0),'end_time']-activity.ix[(person_id,0),'begin_time']
            row_index += 1
       
        # Second activity
        activity.ix[(person_id,row+row_index),'person_id'] = person_id
        activity.ix[(person_id,row+row_index),'activity'] = current_row['dpurp']
        activity.ix[(person_id,row+row_index),'activity_index'] = row+row_index
        activity.ix[(person_id,row+row_index),'parcel'] = next_row['opcl']
        activity.ix[(person_id,row+row_index),'begin_time'] = current_row['arrtm']
        activity.ix[(person_id,row+row_index),'end_time'] = next_row['deptm'] # minutes after midnight
        activity.ix[(person_id,row+row_index),'duration'] = activity.ix[(person_id,row+row_index),'end_time']-activity.ix[(person_id,row+row_index),'begin_time']
            
        # for last trip of the day
        if row == len(trip_subsample)-2:
            activity.ix[(person_id,row+2),'activity_index'] = row+2
            activity.ix[(person_id,row+2),'activity'] = next_row['dpurp']
            activity.ix[(person_id,row+2),'person_id'] = person_id
            activity.ix[(person_id,row+2),'parcel'] = next_row['dpcl']
            activity.ix[(person_id,row+2),'begin_time'] = next_row['arrtm'] # minutes after midnight
            activity.ix[(person_id,row+2),'end_time'] = 1440 # minutes after midnight
            activity.ix[(person_id,row+2),'duration'] = 1440-activity.ix[(person_id,row+2),'begin_time']

In [235]:
# further separate each activity to 12 time periods
# Probably need 15 minutes increments
activity['begin_hour'] = np.round(activity['begin_time']/60.0).astype('int')
activity['end_hour'] = np.round(activity['end_time']/60.0).astype('int')

In [236]:
activity.to_csv('c:/users/brice/person_activity.csv',index=False)

In [268]:
# Split activities into equal 24 hour portions
hourly_activity = pd.DataFrame()
hourly_activity.index = [[],[]]
person_list = pd.unique(activity['person_id']) # temporarily reduce list zize
for person_id in person_list:
    activity_list = activity[activity['person_id'] == person_id]
    for row in xrange(len(activity_list)):
        activity_row = activity_list.iloc[row]
#         print '---------'
        for hour in xrange(activity_row.begin_hour,activity_row.end_hour+1):
#             print hour
            hourly_activity.ix[(activity_row['person_id'],hour),'hour'] = hour
            hourly_activity.ix[(activity_row['person_id'],hour),'parcel'] = activity_row.parcel
            hourly_activity.ix[(activity_row['person_id'],hour),'person_id'] = activity_row.person_id
            
            # Add a percent of hour at this location field

In [170]:
# Look up hourly exposure rates from parcel-based table; multiply by time spent at location


In [62]:
# Bufferd parcels (500ft) intersected with (AM) network componenets for 2014
df = pd.read_csv(r'R:/aq/parcel_network_2014_new.txt')
# Work with a subsample for now
# df = df.iloc[:100000]

In [201]:
# Lookup table for hourly emissions per link
#### for now, make this up
# import pysal as ps
# import numpy as np
# links = ps.open(r'R:\SoundCast\Inputs\2014\networks\edges_0.dbf')
# d = dict([(col, np.array(links.by_col(col))) for col in links.header])
# links = pd.DataFrame(d)

In [54]:
# Join with hourly network results
network_results = pd.read_csv(r'L:\T2040\soundcast_2014\outputs\network\network_results.csv')
# Multiple results from past runs sometimes included, drop those
network_results = network_results.drop_duplicates(subset=['ij','tod'])

In [63]:
# List of volume near each affected parcel for each time period
parcel_vol = pd.merge(df,network_results,left_on=['NewINode','NewJNode'],right_on=['i','j'],how='left')

In [202]:
# Reshape volumes to be for all 24 hours
# dictionary to relate time period to 24 hours
tod_dict = {0: '20to5',
    1: '20to5',
    2: '20to5',
    3: '20to5',
    4: '20to5',
    5: '5to6',
    6: '6to7',
    7: '7to8',
    8: '8to9',
    9: '9to10',
    10: '10to14',
    11: '10to14',
    12: '10to14',
    13: '10to14',
    14: '14to15',
    15: '15to16',
    16: '16to17',
    17: '17to18',
    18: '18to20',
    19: '18to20',
    20: '20to5',
    21: '20to5',
    22: '20to5',
    23: '20to5'
}

In [269]:
hourly_activity['tod'] = hourly_activity['hour'].astype('int')
hourly_activity.replace({"tod":tod_dict},inplace=True)

In [270]:
# type(activity['parcel'].iloc[0])
hourly_activity['parcel'] = hourly_activity['parcel'].astype('int')

In [271]:
# Drop parcerls that don't show up in hourly_activity file to reduce file size
parcel_vol_trimmed = parcel_vol[parcel_vol['parcelid'].isin(pd.unique(hourly_activity.parcel))]

In [272]:
# _df = pd.merge(hourly_activity,parcel_vol_trimmed,left_on=['parcel','tod'],right_on=['parcelid','tod'], how='left')
_df = pd.merge(hourly_activity,parcel_vol_trimmed,left_on=['parcel','tod'],right_on=['parcelid','tod'])
_df['hour'] = _df.hour.astype('int')

In [276]:
# Add in the hourly rates to compute total emissions accumulated over time

# Load some test rates
rates = pd.read_csv(r':\T2040\soundcast_2014\outputs\aq_2014_july.csv')

# Need to use real MOVES rates here: 
# e.g., L:\T2040\soundcast_2014\scripts\summarize\inputs\network_summary\emission_rates_2014.csv

In [291]:
# rates.hourId

In [292]:
newdf = pd.merge(_df,rates,left_on=['hour','ij'], right_on=['hourId','ij'], how='left')
newdf = pd.merge(_df,rates,left_on=['hour','ij'], right_on=['hourId','ij'])

In [299]:
# Not sure what the columns mean for now; just test
newdf['pm_10'] = newdf['total_volume']*newdf['100']

In [300]:
newdf.groupby('person_id').sum()['pm_10']

person_id
101_1      85.547061
101_6       7.389015
104_2      91.797819
105_1     163.061594
107_2      65.964929
108_1      16.232875
108_2       8.986246
114_2      83.287750
116_2     198.049382
118_2     403.284163
121_2     226.369240
123_1     199.593877
124_2     215.516335
126_2     486.689488
132_2     706.849239
133_1       1.798610
134_1     165.017121
135_4     116.121462
136_1     241.844457
141_1    1163.870077
142_1     992.181418
143_1     511.212682
145_1      32.967992
147_1     122.984420
150_1     527.829551
154_1     136.529126
155_1      46.039257
157_1      24.010563
161_1    1225.136012
163_1     498.198629
            ...     
44_1       42.159348
45_1     2694.863909
46_1       34.557095
47_1       65.108869
48_1       16.726594
4_1       199.426815
50_1      207.132538
50_2      257.072613
54_1     2624.457955
56_1      542.349707
58_1       32.972852
5_1        74.984174
61_1      261.368021
65_4      313.045092
66_1      124.764212
66_5      198.306863
66_

In [243]:
# By attaching this to the person record we can analyze results by household, workplace location,
# worker type, income, (equity geographies)